## UW Pharmacy Student Self-Care Analysis (winter 7_8)

This notebook is a draft using the "winter 7_8" page in the student comment data of the 2022-2023 SY in the UW SoP. The analysis roughly looks at the determinants of the categories of self-care students chose for this quarter using VADER and NLTK sentiment analysis and Pandas table manipulation.

Note: this notebook relies on uploading a single csv sheet for one quarter, along with adding two additional columns called "Category 1" and "Category 2" which categorize each comment based on the 8 facets of self-care.

### Setup

In [1]:
#import python libraries
import numpy as np
import pandas as pd

!pip install vaderSentiment
!pip install --upgrade pip
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import spacy

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-01 05:11:38.799205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [2]:
# Can change the file csv right here
file_csv = "winter7_8.csv"

data = pd.read_csv(file_csv)

first_column = data.columns[1]
data = data[[first_column, "Category 1", "Category 2"]]

In [3]:
all_dups = pd.DataFrame()
for i in range(len(data)):
    if not pd.isnull(data["Category 2"][i]):
        temp = data[[first_column, "Category 2"]].iloc[i]
        all_dups = all_dups.append(temp)
    else:
        continue
all_dups = all_dups[[first_column, "Category 2"]]
all_dups = all_dups.rename(columns={"Category 2": "Category"})

### Working data

In [4]:
data = data.rename(columns={"Category 1": "Category"})
data = data[[first_column, "Category"]]
data = data.append(all_dups)
data

,Win 7_8,Category
0,Called a friend who happened to be on her lunc...,Community
1,"For my self care, I went on a walk and listene...",Emotional
2,"Today, I spent my break eating good food and w...",Physical
3,I worked on my Paint by Diamond while listenin...,Mental
4,I walked around the neighborhood for thirty mi...,Physical
...,...,...
100,I went out with friends for dinner and spent t...,Mental
103,I went grocery shopping with my girlfriend,Environmental
104,I took a half hour walk to Gas Works Park,Physical
105,This week for self care I worked on developing...,Occupational


### Category frequencies within student comments

The comments data is displayed here in terms of each of the 8 self-care categories (physical, mental, community, emotional, environmental, spiritual, and occupational) through a pie chart. 

In [5]:
import plotly.express as px

dfg = data.groupby("Category").count().sort_values(by=first_column, ascending=False)
print("All reflections processed (includes duplicate categories): ", np.sum(dfg[first_column]))

dfg_pie = px.pie(dfg.reset_index(), values='Win 7_8', names='Category', title='Category Frequencies')
dfg_pie

All reflections processed (includes duplicate categories):  167


### Sentiment analysis (positivity polarity score for each comment)

In this section, I scored each of the comments using the VADER sentiment analysis function. The function sentiment_scores(sentence) will return the positivity polarity score for each comment probabilistically between -1 and 1, with -1 being the most negative and 1 being the most positive. Note that this model is untrained and will be a very rough interpretation of the comments, i.e. not trained on what self-care is as a whole but on the general tone of each comment.

In [6]:
# function to print sentiments
# of the sentence (vader)

#taken from geekforgeeks
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    return sentiment_dict['compound']

In [7]:
scores = pd.DataFrame(columns={"Win 1_2", "Score"})
for i in data[first_column]:
    sentence = str(i)
    score = sentiment_scores(sentence)
    scores = scores.append({first_column: sentence, "Score": score}, ignore_index=True)

In [8]:
scores_categories = data.merge(scores, on=first_column, how="right")
scores_categories

,Win 7_8,Category,Score,Win 1_2
0,Called a friend who happened to be on her lunc...,Community,0.1779,NaN
1,"For my self care, I went on a walk and listene...",Emotional,0.5411,NaN
2,"For my self care, I went on a walk and listene...",Physical,0.5411,NaN
3,"Today, I spent my break eating good food and w...",Physical,0.4926,NaN
4,"Today, I spent my break eating good food and w...",Mental,0.4926,NaN
...,...,...,...,...
283,I took a half hour walk to Gas Works Park,Physical,0.0000,NaN
284,This week for self care I worked on developing...,Mental,0.9042,NaN
285,This week for self care I worked on developing...,Occupational,0.9042,NaN
286,Went to get breakfast and relax in the morning...,Physical,0.4404,NaN


In [9]:
fig = px.scatter(scores_categories, y="Score", x="Category", title="Polarity Scores by Category")
mean_scores = scores_categories.groupby('Category').mean()
for c in mean_scores.index:
    fig.add_scatter(x=[c],
                    y=[mean_scores.loc[c]['Score']],
                    marker=dict(
                        color='red',
                        size=10
                    ),
                name=f'{c} mean')

fig.show()

In [10]:
px.bar(mean_scores.reset_index().sort_values(by="Score", ascending=False), x="Category", y="Score", title="Average Scores per Category")

### Scores of each category 

In [11]:
print('Physical category table')
scores_categories_physical = scores_categories[scores_categories["Category"] == "Physical"]
scores_categories_physical

Physical category table


,Win 7_8,Category,Score,Win 1_2
2,"For my self care, I went on a walk and listene...",Physical,0.5411,NaN
3,"Today, I spent my break eating good food and w...",Physical,0.4926,NaN
7,I walked around the neighborhood for thirty mi...,Physical,0.7269,NaN
10,This week I focused on Resilience and coping s...,Physical,-0.2484,NaN
13,I went for a walk around my neighborhood,Physical,0.0000,NaN
...,...,...,...,...
268,Go to the gym.,Physical,0.0000,NaN
270,I took a walk around my apartment with my room...,Physical,0.0000,NaN
272,I went to the gym and took my car to the shop.,Physical,0.0000,NaN
283,I took a half hour walk to Gas Works Park,Physical,0.0000,NaN


In [12]:
print('Mental category table')
scores_categories_mental = scores_categories[scores_categories["Category"] == "Mental"]
scores_categories_mental

Mental category table


,Win 7_8,Category,Score,Win 1_2
4,"Today, I spent my break eating good food and w...",Mental,0.4926,NaN
5,I worked on my Paint by Diamond while listenin...,Mental,0.3400,NaN
9,This week I focused on Resilience and coping s...,Mental,-0.2484,NaN
11,Create: I spent some time learning how to cook...,Mental,0.2960,NaN
23,"For my self-care, I spent my time treating mys...",Mental,0.9685,NaN
25,"I went on a photoshoot with a few friends, my ...",Mental,0.7500,NaN
26,[photo] playing game of Jenga,Mental,0.2023,NaN
31,I spent 30 minutes to watch team USA ski and s...,Mental,0.0000,NaN
37,I watched a YouTube video of how to prepare a ...,Mental,0.7351,NaN
54,I played piano for 30 minutes.,Mental,0.3400,NaN


In [13]:
print('Community category table')
scores_categories_community = scores_categories[scores_categories["Category"] == "Community"]
scores_categories_community

Community category table


,Win 7_8,Category,Score,Win 1_2
0,Called a friend who happened to be on her lunc...,Community,0.1779,NaN
12,I'm getting lunch with my dad!,Community,0.0000,NaN
17,I went hiking with a good friend of mine and m...,Community,0.9324,NaN
21,Went skiing with friends on the weekend.,Community,0.4767,NaN
24,"I went on a photoshoot with a few friends, my ...",Community,0.7500,NaN
...,...,...,...,...
271,I took a walk around my apartment with my room...,Community,0.0000,NaN
274,This week I made a puzzle with my friends and ...,Community,0.8316,NaN
276,This week a couple of my classmates had dinner...,Community,0.5413,NaN
278,I went out with friends for dinner and spent t...,Community,0.7430,NaN


In [14]:
print('Emotional category table')
scores_categories_emotional = scores_categories[scores_categories["Category"] == "Emotional"]
scores_categories_emotional

Emotional category table


,Win 7_8,Category,Score,Win 1_2
1,"For my self care, I went on a walk and listene...",Emotional,0.5411,NaN
6,I worked on my Paint by Diamond while listenin...,Emotional,0.3400,NaN
64,Rest/relax I spent time listening to music and...,Emotional,0.0000,NaN
69,I spent 43 minutes to listen to a podcast and ...,Emotional,0.0000,NaN
78,I took a short nap then cooked a meal with my ...,Emotional,0.5106,NaN
81,Create: got reacquainted with and played my sa...,Emotional,0.5423,NaN
91,After class I went on a nice walk by the water...,Emotional,0.4215,NaN
168,"For my self care, I went on a walk and listene...",Emotional,0.5411,NaN
173,I worked on my Paint by Diamond while listenin...,Emotional,0.3400,NaN
209,I spent 43 minutes to listen to a podcast and ...,Emotional,0.0000,NaN


In [15]:
print('Spiritual category table')
scores_categories_spiritual = scores_categories[scores_categories["Category"] == "Spiritual"]
scores_categories_spiritual

Spiritual category table


,Win 7_8,Category,Score,Win 1_2
71,I took a walk at Green Lake park. I went to a ...,Spiritual,0.3182,NaN
151,This week a couple of my classmates had dinner...,Spiritual,0.5413,NaN
211,I took a walk at Green Lake park. I went to a ...,Spiritual,0.3182,NaN
277,This week a couple of my classmates had dinner...,Spiritual,0.5413,NaN


In [16]:
print('Occupational category table')
scores_categories_occupational = scores_categories[scores_categories["Category"] == "Occupational"]
scores_categories_occupational

Occupational category table


,Win 7_8,Category,Score,Win 1_2
98,I went and got a quick lunch with a friend bef...,Occupational,0.4939,NaN
104,"Over the three-day weekend, I invited friends ...",Occupational,0.8899,NaN
127,Sometimes things slip away and I get overwhelm...,Occupational,0.7184,NaN
133,I'm practicing a nurture connection via social...,Occupational,0.8591,NaN
138,I tried to finish my note for one class then w...,Occupational,0.4019,NaN
164,This week for self care I worked on developing...,Occupational,0.9042,NaN
233,I went and got a quick lunch with a friend bef...,Occupational,0.4939,NaN
239,"Over the three-day weekend, I invited friends ...",Occupational,0.8899,NaN
261,I'm practicing a nurture connection via social...,Occupational,0.8591,NaN
266,I tried to finish my note for one class then w...,Occupational,0.4019,NaN


### Frequency of nouns and verbs 

In [17]:
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from collections import Counter
import en_core_web_sm

2023-06-01 05:11:59.979577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 05:12:00.253410: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 05:12:00.253454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-01 05:12:00.292386: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 05:12:02.041416: W tensorflow/stream_executor/platform/de

Note: Just for the sake of experimentation, I only did this part on comments categorized as "physical." Can easily broaden it to more categories if useful.

This section takes the top 100 ('top' meaning most frequent) words from a particular category along with its count within the comments. Again, note that there may be duplicates depending on if the comment was repeated if it was categorized twice. I then created a function to extract nouns and verbs within that category. This can potentially be helpful if we were to see which action within each category students tended to choose.

In [18]:
def fig_frequencies(tbl, keyword):
    counter = Counter(" ".join(tbl[first_column]).split()).most_common(100)
    array = []
    for c in counter:
        array = np.append(array, c[0])

    sentence = ','.join(array)

    def extract_nouns_verbs(sentence):
        text = nltk.word_tokenize(sentence)
        pos_tagged = nltk.pos_tag(text)
        nouns_verbs = filter(lambda x:x[1]=='NN' or x[1] == 'VB',pos_tagged)
        return list(nouns_verbs)

    nouns_verbs = pd.DataFrame(extract_nouns_verbs(sentence), columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency = nouns_verbs.merge(counted_words, on='Word', how='left').sort_values(by='Word frequency', ascending=False)

    nlp = en_core_web_sm.load()
    def all_nouns(sentence):
        doc = nlp(sentence)
        nouns = [(token.lemma_, "NN") for token in doc if token.pos_ == "NOUN"]
        return nouns

    def all_verbs(sentence):
        doc = nlp(sentence)
        verbs = [(token.lemma_, "VB") for token in doc if token.pos_ == "VERB"]
        return verbs

    nouns_verbs_spacy = all_nouns(sentence) + all_verbs(sentence)

    nouns_verbs_sp = pd.DataFrame(nouns_verbs_spacy, columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency_sp = nouns_verbs_sp.merge(counted_words, on='Word', how='inner').sort_values(by='Word frequency', ascending=False)
    noun_verb_frequency_sp = noun_verb_frequency_sp.drop_duplicates(subset='Word', keep="last")

    fig = px.bar(noun_verb_frequency_sp, x='Word', y='Word frequency', color='Word type', title='Noun/verb frequencies for Category: ' + keyword)
    fig.update_layout(xaxis_categoryorder = 'total descending')

    fig.show()

In [19]:
fig_frequencies(scores_categories_physical, 'Physical');

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning:

[W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.



In [20]:
fig_frequencies(scores_categories_mental, 'Mental');

In [21]:
fig_frequencies(scores_categories_emotional, 'Emotional');

In [22]:
fig_frequencies(scores_categories_community, 'Community');

In [23]:
fig_frequencies(scores_categories_spiritual, 'Spiritual');

In [24]:
fig_frequencies(scores_categories_occupational, 'Occupational');

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443b2d8b-ed93-43a6-bbe3-c3fe69ac80e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>